names目录下中存储了1880到2020年期间美国新生儿的姓名、出生年份、性别、出现次数等信息。请使用pandas处理数据，文件以txt形式存储，完成以下任务：

1）统计数据集中男孩名字和女孩名字各有多少种；

2）将数据集按照name字段进行分组，然后求和，赋值给变量names，最后输出前五行结果；

3）按照每个名字被使用的次数（count）对第⼆步中结果进行降序排序，得出最受欢迎的五个名字；

4）在数据集中，共出现了多少个不同的名字？（不包含重复项）

In [1]:
import pandas as pd
import os

# 设置文件路径
file_directory = './names/'
years = range(1880, 2021)  # year数据从1880-2020年
all_data = []

# 循环遍历所有文件，读取，并添加到一个列表中
for year in years:
    file_path = os.path.join(file_directory, f'yob{year}.txt')
    
    # 利用pandas定义列属性名
    df_year = pd.read_csv(file_path, names=["Name", "Gender", "Count"])
    df_year['Year'] = year  # 另外添加一列year属性
    all_data.append(df_year)

# 将所有数据整合到同一个dataframe
df = pd.concat(all_data, ignore_index=True)

# 1) 统计男孩和女孩的名字各有多少种
num_boy_names = df[df['Gender'] == 'M']['Name'].nunique()
num_girl_names = df[df['Gender'] == 'F']['Name'].nunique()

# 2) 按name字段进行分组
names = df.groupby("Name").agg({"Count": "sum"}).reset_index()

# 3) Sort by usage count and get the top 5 most popular names
most_popular_5 = names.sort_values(by="Count", ascending=False).head()

# 4) Get the total number of unique names
unique_names_count = df['Name'].nunique()

print(f'男生名字有{num_boy_names}种,女生名字有{num_girl_names}种')
print(f'前五行的名字\n{names.head()}')
print(f'前五多的名字\n{most_popular_5}')
print(f'共有{unique_names_count}个不同的名字')


男生名字有42567种,女生名字有68905种
前五行的名字
      Name  Count
0    Aaban    120
1    Aabha     46
2    Aabid     16
3  Aabidah      5
4    Aabir     10
前五多的名字
          Name    Count
39948    James  5213689
44359     John  5163958
77413   Robert  4849738
65094  Michael  4405274
95980  William  4159868
共有100364个不同的名字


breast-cancer-wisconsin.data是⼀个医学数据集，包含了乳腺癌患者的诊断结果和相关指标，如：肿块大小、形状、边缘、质地、细胞核大小、细胞核形状等。该数据集共有11列，第1列为id，第2-10列为特征，第11列为标签（2为良性、4为恶性）。基于该数据集，完成以下任务：

1）⼿动实现逻辑回归（除数据处理工具外不允许直接调用其他逻辑回归算法库）来进⾏癌症分类预测，具体实现方式可用梯度下降法；

2）使用工具库，同样实现逻辑回归，并与手动实现的结果进行比较。

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# 第一步：加载和预处理数据
# 数据文件路径
file_path = './breast-cancer-wisconsin.data'

# 定义数据集的列名
columns = ['id', 'clump_thickness', 'cell_size_uniformity', 'cell_shape_uniformity', 
           'marginal_adhesion', 'epithelial_cell_size', 'bare_nuclei', 
           'bland_chromatin', 'normal_nucleoli', 'mitoses', 'class']

# 读取数据
df = pd.read_csv(file_path, names=columns)

# 替换缺失值（'?'）为NaN，然后丢弃含有NaN的行
df.replace('?', np.nan, inplace=True)
df.dropna(inplace=True)

# 将'bare_nuclei'列转换为整数类型（原始数据中由于存在缺失值导致其为字符串类型）
df['bare_nuclei'] = df['bare_nuclei'].astype(int)

# 分离特征和标签
X = df.iloc[:, 1:-1].values  # 特征（第2-10列）
y = df['class'].values  # 标签（第11列）

# 将标签转换为二进制（2代表良性，转换为0；4代表恶性，转换为1）
y = np.where(y == 2, 0, 1)

# 标准化特征数据
X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)

# 添加偏置项（在X的第一列添加全为1的列）
X = np.hstack([np.ones((X.shape[0], 1)), X])

# 将数据划分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 第二步：手动实现逻辑回归
# 初始化逻辑回归的参数（权重和偏置）
def sigmoid(z):
    """定义sigmoid函数"""
    return 1 / (1 + np.exp(-z))

def compute_cost(X, y, theta):
    """定义损失函数（交叉熵损失）"""
    m = len(y)
    h = sigmoid(np.dot(X, theta))  # 预测值
    cost = -(1/m) * np.sum(y * np.log(h) + (1 - y) * np.log(1 - h))  # 计算损失
    return cost

def gradient_descent(X, y, theta, alpha, num_iters):
    """梯度下降算法"""
    m = len(y)
    cost_history = []

    for i in range(num_iters):
        # 计算预测值
        h = sigmoid(np.dot(X, theta))
        # 更新权重
        theta -= (alpha / m) * np.dot(X.T, h - y)
        # 记录每次迭代的损失值
        cost = compute_cost(X, y, theta)
        cost_history.append(cost)
    return theta, cost_history

# 初始化参数
theta = np.zeros(X_train.shape[1])  # 初始化theta为0
alpha = 0.01  # 学习率
num_iters = 100  # 迭代次数

# 使用梯度下降算法训练模型
theta_final, cost_history = gradient_descent(X_train, y_train, theta, alpha, num_iters)

# 定义预测函数
def predict(X, theta):
    """预测函数：返回0或1"""
    return np.round(sigmoid(np.dot(X, theta)))

# 使用手动实现的逻辑回归模型进行预测
y_pred_manual = predict(X_test, theta_final)

# 计算手动实现的逻辑回归模型的准确率
accuracy_manual = np.mean(y_pred_manual == y_test) * 100
print(f"手动实现逻辑回归的准确率: {accuracy_manual:.2f}%")

# 第三步：使用工具库实现逻辑回归，并进行比较
# 使用scikit-learn的LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train[:, 1:], y_train)  # 注意：工具库不需要偏置项，因此不包含X的第一列

# 进行预测
y_pred_sklearn = logreg.predict(X_test[:, 1:])

# 计算scikit-learn实现的准确率
accuracy_sklearn = accuracy_score(y_test, y_pred_sklearn) * 100
print(f"使用scikit-learn的逻辑回归准确率: {accuracy_sklearn:.2f}%")


手动实现逻辑回归的准确率: 97.56%
使用scikit-learn的逻辑回归准确率: 95.61%
